![Velomatch](resources/velomatch.png)

# Stock Broker Example

This notebook demonstrates how to send stock data to the Velomatch Stock Broker.



## Generating

### JSON

In [ ]:
json_stock_data_set = {
    "username": "DistributerFoo",
    "password": "PasswordBar",
    "datetime": "2021-11-24T14:40:40+0000",
    "facility": [
        {
            "facilitycode": "f001",
            "partial": False,
            "stock": { 
                "unit": [
                    {
                        "sku": "123-456-789",
                        "quantity": 4,
                    },
                    {
                        "sku": "223-456-789",
                        "quantity": 7,
                    },
                    {
                        "sku": "324-456-789",
                        "quantity": 0,
                    },
                ]
             },
            "forecast": { 
                "forecastid": "f1001",
                "datetime": "2021-11-26T14:40:40+0000",
                "stock": { 
                    "unit": [
                        {
                            "sku": "324-456-789",
                            "quantity": 3,
                        },
                    ]
                },
            }, 
        },
        {
            "facilitycode": "f002", 
            "partial": True,
            "stock": { 
                "unit": [
                    {
                        "sku": "423-456-789",
                        "quantity": 3,
                    },
                    {
                        "sku": "523-456-789",
                        "quantity": 6,
                    },
                    {
                        "sku": "624-456-789",
                        "quantity": 2,
                    },
                ]
             },
            "forecast": [ 
                { 
                    "forecastid": "f1002",
                    "datetime": "2021-11-28T14:40:40+0000",
                    "stock": { 
                        "unit": [
                            {
                                "sku": "624-456-789",
                                "quantity": 2,
                            },
                        ]
                    },
                }, 
                { 
                    "forecastid": "f1003",
                    "datetime": "2021-12-01T14:40:40+0000",
                    "stock": { 
                        "unit": [
                            {
                                "sku": "423-456-789",
                                "quantity": 1,
                            },
                            {
                                "sku": "524-456-789",
                                "quantity": 3,
                            },
                        ]
                    },
                }, 
            ]
        }
    ]
}

import pprint
pp = pprint.PrettyPrinter(width=41, compact=False)
pp.pprint(json_stock_data_set)

### XML

In [ ]:

xml_stock_data_set = "<?xml version=\"1.0\" encoding=\"ISO-8859-1\" ?>" \
    "<Velomatch>" \
        "<StockBroker username=\"DistributerFoo\" password=\"PasswordBar\" DateTime=\"2021-11-24T14:40:40+0000\">" \
            "<Facilities>" \
                "<Facility facilityCode=\"f001\" partial=\"false\">" \
                    "<Stock>" \
                        "<Unit sku=\"123-456-789\" quantity=\"4\" />" \
                        "<Unit sku=\"223-456-789\" quantity=\"7\" />" \
                        "<Unit sku=\"323-456-789\" quantity=\"6\" />" \
                    "</Stock>" \
                    "<Forecasts>" \
                        "<Forecast forecastid=\"f1001\" datetime=\"2021-11-25T14:40:40+0000\">" \
                            "<Stock>" \
                                "<Unit sku=\"324-456-789\" quantity=\"4\" />" \
                            "</Stock>" \
                        "</Forecast>" \
                        "<Forecast id=\"f1002\" datetime=\"2021-11-26T14:40:40+0000\">" \
                            "<Stock>" \
                                "<Unit sku=\"423-456-789\" quantity=\"4\" />" \
                                "<Unit sku=\"523-456-789\" quantity=\"7\" />" \
                                "<Unit sku=\"623-456-789\" quantity=\"6\" />" \
                            "</Stock>" \
                        "</Forecast>" \
                    "</Forecasts>" \
                "</Facility>" \
            "</Facilities>" \
        "</StockBroker>" \
    "</Velomatch>"

import xml.dom.minidom
dom = xml.dom.minidom.parseString(xml_stock_data_set)
print(dom.toprettyxml())


### CSV

In [ ]:
csv_stock_data_set = "username,password,datetime\n" \
    "\"DistributerFoo\",\"PasswordBar\",2021-11-24T14:40:40+0000\n" \
    "facilitycode,sku,quantity,datetime,quantity,datetime,quantity\n" \
    "\"f001\",\"123-456-789\",4\n" \
    "\"f001\",\"223-456-789\",7\n" \
    "\"f001\",\"323-456-789\",6\n" \
    "\"f001\",\"324-456-789\",0,2021-11-25T14:40:40+0000,4,2021-11-29T10:00:40+0000,2\n" \
    "\"f001\",\"423-456-789\",0,2021-11-25T14:40:40+0000,4\n" \
    "\"f001\",\"523-456-789\",0,2021-11-25T14:40:40+0000,7\n" \
    "\"f001\",\"623-456-789\",0,2021-11-25T14:40:40+0000,6\n"

print (csv_stock_data_set)


## Sending

### SFTP

The example below shows sending three separate files, one for JSON, XML and CSV respectively. Note, it is only necessary to send one of these file types. 

In [ ]:
import sys
!{sys.executable} -m pip install pysftp

import pysftp
import os
import json

from datetime import datetime

today = datetime.today()

dateTime = today.strftime("%Y%m%d-%H%M%S")

sftphostname = "151.236.222.88"
sftpUsername = "test"
sftpPassword = "RIIVe!OM@tch@2021"
sftpDirectory = "/" + sftpUsername

json_stock_file = dateTime + "-" + sftpUsername + ".json"
with open(json_stock_file, 'w') as f:
    f.write(json.dumps(json_stock_data_set))

xml_stock_file = dateTime + "-" + sftpUsername + ".xml"
with open(xml_stock_file, 'w') as f:
    f.write(xml_stock_data_set)

csv_stock_file = dateTime + "-" + sftpUsername + ".csv"
with open(csv_stock_file, 'w') as f:
    f.write(csv_stock_data_set)

with pysftp.Connection(sftphostname, username=sftpUsername, password=sftpPassword) as sftp:

    with sftp.cd(sftpDirectory):        # change directory
        sftp.put(json_stock_file)  	# upload file on remote
        sftp.put(xml_stock_file)  	# upload file on remote
        sftp.put(csv_stock_file)  	# upload file on remote


os.remove(json_stock_file)
os.remove(xml_stock_file)
os.remove(csv_stock_file)

## About Velomatch

Cycling is glorious! The buying process often isn't... We're here to change that. Unbiased, expert advice without the hard sell. Make sure your next bike fits! Visit www.velomatch.com to try it out for yourself. 